In [2]:
import os
import sys
import pandas as pd
import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# NAVER BLOG Crawling
- title/url 

In [ ]:
# 검색어 입력
query = input("검색어 : ")


# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

print(startDate, endDate)


# Chrome driver 환경설정 및 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)
time.sleep(1)


# 총 검색 결과
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
print("총 블로그글 개수 : ", search_number.text)


# 연관 검색어 
sub_keywords_data = soup.select('div.area_keyword > div.list')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())

print("연관검색어:",*sub_keywords)


# blog url/title 가져오기
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
page_num = 1
end_page=total_num//7+1
contents_num = 7

blog_title_lst = []
blog_url_lst = []

while contents_num == 7 : 
    search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(search_url)
    time.sleep(0.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # 블로그 제목, url, 포스팅 날짜 가져오기
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')

    for post in posts_data :
        title=post.get_text().replace('\n','').strip()
        href=post.attrs['href']
        blog_title_lst.append(title)
        blog_url_lst.append(href)
    
    page_num +=1
    contents_num = len(posts_data)

print(blog_title_lst)
print(blog_url_lst)

In [ ]:
# 확인
print(len(blog_title_lst),len(blog_url_lst))

- blog 본문 가져오기(test)

In [ ]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)


# 해당 blog 불러오기 
url = blog_url_lst[0]
driver.get(url)
time.sleep(1)


# iframe 접근 
driver.switch_to.frame('mainFrame')


# 제목 가져오기
overlays_title = ".se-module.se-module-text.se-title-text"
html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
title = html_title.text
print(title)


# 포스팅 날짜 가져오기
overlays_publishDate = ".se_publishDate.pcol2"
html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
date = html_publishDate.text
print(date)

# 본문 가져오기 
overlays_content = ".se-component.se-text.se-l-default"
html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
contents_lst = []
for content in html_content : 
    contents_lst.append(content.text.replace('\n',''))
    

print(contents_lst)

# 해시태그 가져오기 

# title = html_title.text 
# title





- blog 본문 크롤링

In [74]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)

crawling_blog_data = []


# blog 본문 가져오기 
for i in range(len(blog_url_lst)) :    
    url = str(blog_url_lst[i])
    driver.get(url)
    time.sleep(1)
    
    
    # 본문 크롤링 예외 처리 
    try :
        # iframe 접근 
        driver.switch_to.frame('mainFrame')

        
        # 제목 가져오기
        overlays_title = ".se-module.se-module-text.se-title-text"
        html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
        title = html_title.text

        
        # 작성 날짜 가져오기
        overlays_publishDate = ".se_publishDate.pcol2"
        html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
        date = html_publishDate.text

        
        # 본문 가져오기
        overlays_content = ".se-component.se-text.se-l-default"
        html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
        contents_lst = []
        for content in html_content : 
            contents_lst.append(content.text.replace('\n',''))
        content_str = ''.join(contents_lst)

        
        # 크롤링한 내용 담기 
        crawling_blog_data.append({"title":title, "date":date, "content":content_str})
        
        # driver 종료 
        driver.close
        time.sleep(1)
        
        
    # 에러 발생시 다음 블로그로 이동
    except : 
        print("error",i,title)
        driver.close()
        time.sleep(1)

driver.close()

In [ ]:
print(crawling_blog_data)

In [ ]:
df1=pd.DataFrame(crawling_blog_data)
df1

In [83]:
file_name = f"{query}_{datetime.now().strftime('%Y%m%d')}.xlsx"
df.to_excel(file_name, index=False)

In [ ]:
df['content'][0]

In [ ]:
df2 = pd.DataFrame(crawling_blog_data).sort_values(by=date, axis=0, ascending=False, inplace=True) 